In [ ]:
import math
 
def getAngle(landmark1,landmark2,landmark3):
    x1,y1 = landmark1
    x2,y2 = landmark2 
    x3,y3 = landmark3
    
    angle = math.degrees(math.atan2(y3-y2,x3-x2)-math.atan2(y1-y2,x1-x2))
    if angle< 0:
        angle = -angle
    if angle > 180:
        angle = 360-angle

    return angle
print(getAngle((0,0),(0,5),(-2,7)))

In [ ]:
def sit_stand_test(key_points):
    right_ankle = (key_points["right Ankle"].x, key_points["right Ankle"].y)
    right_knee = (key_points["right Knee"].x, key_points["right Knee"].y)
    right_hip= (key_points["right Hip"].x,key_points["right Hip"].y)
    left_ankle = (key_points["left Ankle"].x,key_points["left Ankle"].y)
    left_knee = (key_points["left Knee"].x, key_points["left Knee"].y)
    left_hip = (key_points["left Hip"].x, key_points["left Hip"].y)
    right_leg_angle = getAngle(right_ankle,right_knee,right_hip)
    left_leg_angle = getAngle(left_ankle,left_knee,left_hip)
    # print(right_leg_angle,left_leg_angle)
    if right_leg_angle>=160 or left_leg_angle>=160:
        return True
    else:
        return False


In [ ]:

import numpy as np
def resize_image(img, target_width=1280, target_height=720):
    original_height, original_width = img.shape[:2]
    ratio = min(target_width/original_width, target_height/original_height)
    
    # Compute new dimensions
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # Resize the original image
    resized_image = cv2.resize(img, (new_width, new_height))

    # Create a black canvas of the target size
    canvas = np.zeros((target_height, target_width, 3), dtype=np.uint8)

    # Compute position to center the image on the canvas
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2

    # Place the resized image onto the center of the canvas
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_image

    return canvas

In [ ]:
import mediapipe as mp
import time
import cv2

mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()
#REPLACE WITH VIDEO FILE NAME PATH BELOW
cap = cv2.VideoCapture('sit_stand.mp4')

pTime = 0
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#REPLACE WITH OUTPUT VIDEO FILE NAME PATH BELOW
output_video = cv2.VideoWriter('sit_stand_final.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (1280,720))


sit_stand_results = False
start_writing = False
count = 0
while cap.isOpened():
    success, img = cap.read()
    if not success:
        break  # Exits the loop if no frames are left to read or if there's an error
    img = resize_image(img)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    
    if results.pose_landmarks:  
        mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
        key_points = {}
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
            if id == 28:
                key_points["right Ankle"]= lm
            elif id == 26:
                key_points["right Knee"]= lm
            elif id == 24:
                key_points["right Hip"]= lm
            elif id == 27:
                key_points["left Ankle"]= lm                
            elif id == 25:
                key_points["left Knee"]= lm
            elif id == 23:
                key_points["left Hip"]= lm

        if sit_stand_test(key_points) == True:
            count+=1

        if count>=20:
            sit_stand_results= True    
            cv2.putText(img, f"Test: Passed", (70, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        else:
            cv2.putText(img, f"Test: Failed", (70, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        start_writing = True
 
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    # cv2.imshow("Image", img)
    if start_writing ==True:
        output_video.write(img)
    if cv2.waitKey(1) & 0xFF == 27:  # Proper place to check for escape condition
        break

cap.release()
output_video.release()
cv2.destroyAllWindows()
